# Multi-factor strategy

In [2]:
from matplotlib import rc
from collections import defaultdict
from typing import Optional

import FinanceDataReader as fdr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

from data.data_loader import PykrxDataLoader

In [3]:
# 한글 폰트 설정
import platform
os_system = platform.system()
# 시스템에 따라 폰트 설정
if os_system == 'Windows':
    font_name = 'Malgun Gothic'
elif os_system == 'Darwin':  # macOS
    font_name = 'AppleGothic'
else:
    font_name = 'Arial'
rc('font', family=font_name)

plt.rcParams['axes.unicode_minus'] = False
eng_to_kor = {'relative': '모멘텀', 'per': 'PER', 'pbr': 'PBR', 'dividend': '배당',
              'small': '소형주', 'lowvol': '로우볼', 'individual': '개인 수급 주체',
              'institutional': '기관 수급 주체', 'foreign': '외국인 수급 주체',
              'cluster': '군집', 'factor': '전략', 'date': '날짜', 'real': '실제',
              'pred': '예측', 'accuracy': '정확도'}

In [4]:
fromdate = '2012-11-01'
todate = '2021-12-30'
#데이터 종류
macro_name = [
    # 주요 지수
    'KS200',  # 코스피 200
    'US500',  # S&P 500 지수
    'SSEC',  # 상해 종합
    'VIX',  # 공포지수

    # 상품 선물
    'CL',  # WTI유 선물 Crude Oil (NYMEX)
    'GC',  # 금 선물 (COMEX)
    'HG=F',  # 구리 선물 (COMEX)

    # 환율
    'KRW/USD',  # 달러 원화
    'KRW/CNY',  # 달러 위엔화

    # 채권
    'US5YT',  # 5년 만기 미국국채 수익률
    'US30YT',  # 30년 만기 미국국채 수익률
    'FRED:T10Y3M',  # 미국 장단기금리차(10Y-3M) : 연준에서 중시하는 10년-3개월 금리차

    # 경기 지표(미국)
    'FRED:M1SL',  # M1 통화량
    'FRED:M2',  # M2 통화량
    'FRED:HSN1F',  # HSN1F 주택판매지수
    'FRED:T5YIFR',  # 5년 기대인플레이션
    'FRED:UNRATE',  # 미국 실업률

    # 경기 지표(한국)
    'FRED:MANMM101KRM189S',  # 대한민국 M1 통화량
    'FRED:MYAGM2KRM189S',  # 대한민국 M2 통화량
    'FRED:KORCPIALLMINMEI',  # 한국 소비자물가지수: 모든 항목
    'FRED:KORLOLITONOSTSAM',  # OECD 선행지수: 대한민국용 정규화된 선행지수
    'FRED:XTEXVA01KRM664S',  # 대한민국 수출: 상품 가치
    'FRED:XTIMVA01KRM667S',  # 대한민국 수입: 상품 가치
]